In [ ]:
Colab Link (IITGN):
https://colab.research.google.com/drive/14jnUU_AnAVP9V8UEYoxHFwsdrHrYqgEQ?usp=sharing

In [1]:
! git clone https://github.com/devvrat-joshi/datafiles
! unzip "/content/datafiles/dataset.zip"

Cloning into 'datafiles'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (9/9), done.
Archive:  /content/datafiles/dataset.zip
   creating: images/
  inflating: images/dogs0.jpg        
  inflating: images/dogs1.jpg        
  inflating: images/dogs10.jpg       
  inflating: images/dogs11.jpg       
  inflating: images/dogs12.jpg       
  inflating: images/dogs13.jpg       
  inflating: images/dogs14.jpg       
  inflating: images/dogs15.jpg       
  inflating: images/dogs16.jpg       
  inflating: images/dogs17.jpg       
  inflating: images/dogs18.jpg       
  inflating: images/dogs19.jpg       
  inflating: images/dogs2.jpg        
  inflating: images/dogs20.jpg       
  inflating: images/dogs21.jpg       
  inflating: images/dogs22.jpg       
  inflating: images/dogs23.jpg       
  inflating: images/dogs24.jpg       

# VGG1

In [3]:

import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
from os import listdir
from numpy import asarray,save
from numpy import save
# define location of dataset
folder = '/content/images/'

photos, labels = list(), list()
for file in listdir(folder):
	output = 0.0
	if file.startswith('jaguar'):
		output = 1.0
	photo = load_img(folder + file, target_size=(200, 200))
	photo = img_to_array(photo)
	photos.append(photo)
	labels.append(output)
    
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)

from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

# create directories
dirMain = '/content/images/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'jaguar/']
	for labldir in labeldirs:
		newdir = dirMain + subdir + labldir
		print(newdir)
		os.makedirs(newdir, exist_ok=True)

import random
dogsImages = []
jaguarImages = []

for file in listdir(folder):
	if file.startswith('dogs'):
		dogsImages.append(file)
	elif file.startswith('jaguar'):
		jaguarImages.append(file)

testDogs = dogsImages[:len(dogsImages)//4]
trainDogs = dogsImages[len(dogsImages)//4:]
testJaguar = jaguarImages[:len(dogsImages)//4]
trainJaguar = jaguarImages[len(dogsImages)//4:]

srcDir = '/content/images/'
dest = 'train/'
for j in range(len(trainDogs)):
	src = srcDir + trainDogs[j]
	dst = dirMain + dest + 'dogs/' + trainDogs[j]
	copyfile(src, dst)
	src = srcDir + trainJaguar[j]
	dst = dirMain + dest + 'jaguar/' + trainJaguar[j]
	copyfile(src, dst)

dest = 'test/'
for j in range(len(testDogs)):
	src = srcDir + testDogs[j]
	dst = dirMain + dest + 'dogs/' + testDogs[j]
	copyfile(src, dst)
	src = srcDir + testJaguar[j]
	dst = dirMain + dest + 'jaguar/' + testJaguar[j]
	copyfile(src, dst)

def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig('VGG1_plot.png')
	pyplot.close()


# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig('VGG1_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = datagen.flow_from_directory('images/train/',class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = datagen.flow_from_directory('images/test/',class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=100, verbose=0)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
 
# entry point, run the test harness
run_test_harness()

(80, 200, 200, 3) (80,)
/content/images/train/dogs/
/content/images/train/jaguar/
/content/images/test/dogs/
/content/images/test/jaguar/
Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


> 85.000


# Data Augmentation VGG1

In [4]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig('dataAugmentationVGG1_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generators
	train_datagen = ImageDataGenerator(rescale=1.0/255.0,
		width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	test_datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = train_datagen.flow_from_directory('./images/train/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = test_datagen.flow_from_directory('./images/test/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=0)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


> 85.000


# Transfer Learning


In [5]:
# vgg16 model used for transfer learning on the dogs and cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(200, 200, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig('TransferLearning_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	# prepare iterator
	train_it = datagen.flow_from_directory('./images/train/',class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = datagen.flow_from_directory('./images/test/',class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=5, verbose=1)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

58892288/58889256 [==============================] - 0s 0us/step
Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5
1/1 [==============================] - 12s 12s/step - loss: 20.2525 - accuracy: 0.4833 - val_loss: 54.6580 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 443ms/step - loss: 54.5471 - accuracy: 0.5000 - val_loss: 4.7085e-19 - val_accuracy: 1.0000
Epoch 3/5
1/1 [==============================] - 0s 436ms/step - loss: 1.1509e-06 - accuracy: 1.0000 - val_loss: 9.4536e-24 - val_accuracy: 1.0000
Epoch 4/5
1/1 [==============================] - 0s 451ms/step - loss: 2.3513e-15 - accuracy: 1.0000 - val_loss: 6.9783e-17 - val_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 434ms/step - loss: 4.4170e-24 - accuracy: 1.0000 - val_loss: 1.9184e-11 - val_accuracy: 1.0000
> 100.000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
